In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# Exploratory Analysis

This section contains the code to satisfy https://github.com/Quantyra/kaggle-ml-pipeline-icr-challenge/issues/5
It processes and cleans the features, prepares them for use in the model.

See https://www.kaggle.com/competitions/icr-identify-age-related-conditions/data for information about the dataset

In [2]:
# Extracts the class column from the dataset and returns it. Then mutates the dataset by dropping the class column. Also ensures that Id column is completely unique and sets it as the index
def extract_classes(dataset):
    assert dataset['Id'].nunique() == dataset.shape[0]
    dataset.index = dataset['Id']
    dataset.drop(['Id'], axis='columns', inplace=True)
    if 'Class' in dataset.columns:
        y = dataset['Class']
        dataset.drop(['Class'], axis='columns', inplace=True)
        return y

In [3]:
# Some checks to ensure sanity of outputs after engineering
def data_integrity_checks(dataset, classes, initial_row_count):
    assert classes.name == 'Class'
    assert classes.shape[0] == dataset.shape[0]
    assert not dataset.isna().any().any() #No more nulls should be remaining as we converted them to zero
    assert dataset.shape[0] == initial_row_count

In [4]:
#Encode the categorical values, but put an error check in place for the test set since it doesn't have the second class in the sample data.
def encode_categories(dataset, column):
    if dataset[column].nunique() > 1:
        dataset[column] = pd.get_dummies(dataset[column], drop_first=True)
    else:
        dataset[column] = 0

In [5]:
#Mark rows that have null values with a binary flag, and reset null values to zeroes.
def mark_missing_observations(dataset):
    for column in dataset.columns:
        new_columns = dataset[column].apply(lambda x: 1 if pd.isna(x) else 0)
        new_columns.name = f"{column}_missing"
        dataset = pd.concat([dataset, new_columns], axis='columns')
        dataset[column] = dataset[column].fillna(0)
        
    return dataset

## Training set engineering

We clean the training set by extracting the classes column, encoding the EJ column as a binary categorical value, and marking those rows/columns that have missing data while replacing null values with zero. Then we do some data integrity checks to ensure we have no null values and sane outputs.

In [6]:
train_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
greeks_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
initial_row_count = train_df.shape[0]
y_train = extract_classes(train_df)
encode_categories(train_df, 'EJ')
train_df = mark_missing_observations(train_df)
train_df = train_df.astype(float)
data_integrity_checks(train_df, y_train, initial_row_count)
print('Summary Tables')
print('-----------')
print(train_df.info())
print('-----------')
print(train_df.describe())

Summary Tables
-----------
<class 'pandas.core.frame.DataFrame'>
Index: 617 entries, 000ff2bfdfe9 to ffcca4ded3bb
Columns: 112 entries, AB to GL_missing
dtypes: float64(112)
memory usage: 544.7+ KB
None
-----------
               AB            AF           AH          AM          AR  \
count  617.000000    617.000000   617.000000  617.000000  617.000000   
mean     0.477149   3502.013221   118.624513   38.968552   10.128242   
std      0.468388   2300.322717   127.838950   69.728226   10.518877   
min      0.081187    192.593280    85.200147    3.177522    8.138688   
25%      0.252107   2197.345480    85.200147   12.270314    8.138688   
50%      0.354659   3120.318960    85.200147   20.533110    8.138688   
75%      0.559763   4361.637390   113.739540   39.139886    8.138688   
max      6.161666  28688.187660  1910.123198  630.518230  178.943634   

               AX          AY          AZ           BC           BD   ...  \
count  617.000000  617.000000  617.000000   617.000000    6

## Test dataset

Upon notebook submission, the test dataset will be replaced by the full competition test dataset. This section of the notebook prepares the test data for use in the model.

In [7]:
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
initial_row_count = test_df.shape[0]
y_test = extract_classes(test_df)
encode_categories(test_df, 'EJ')
test_df = mark_missing_observations(test_df)
test_df = test_df.astype(float)
print("Summary Tables")
print("______________")
print(test_df.info())
print("______________")
print(test_df.describe())

Summary Tables
______________
<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 00eed32682bb to 046e85c7cc7f
Columns: 112 entries, AB to GL_missing
dtypes: float64(112)
memory usage: 4.4+ KB
None
______________
        AB   AF   AH   AM   AR   AX   AY   AZ   BC  BD   ...  FI_missing  \
count  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  ...         5.0   
mean   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
std    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
min    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
25%    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
50%    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
75%    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   
max    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...         0.0   

       FL_missing  FR_missing  FS_missing  GB_missing  GE_missing  GF_missing  \
count         5.0

## Comparing Two CSV Files

This section contains the code to satisfy https://github.com/Quantyra/kaggle-ml-pipeline-icr-challenge/issues/3. It takes two CSV files and compares them to see if the final submission is properly formatted to the sample_submission.csv file. The format must contain a header and follow the same format as the submission which includes three columns and two different data types (object, and float64).

Sample Submission File:

```
Id,class_0,class_1
00eed32682bb,0.5,0.5
010ebe33f668,0.5,0.5
02fa521e1838,0.5,0.5
040e15f562a2,0.5,0.5
046e85c7cc7f,0.5,0.5
```

In [8]:
def compare_csv_formats(file1, file2):
    """Compares the format of two CSV files using Pandas and returns a boolean.

    Args:
    file1: The path to the first CSV file.
    file2: The path to the second CSV file.

    Returns:
    A boolean indicating whether the format of the two CSV files are identical.
    """
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    if list(df1.columns) != list(df2.columns):
        return False

    if df1.shape[1] != df2.shape[1]:
        return False

    if not df1.dtypes.equals(df2.dtypes):
        return False

    return True